In [10]:
%pip install -U numpy tqdm

Note: you may need to restart the kernel to use updated packages.


# Part One

In [11]:
import numpy as np

with open("input.txt", "r") as file:
    input = file.read().split("\n")

input = np.array([list(string) for string in input])

input

array([['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ...,
       ['#', '.', '#', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.']], shape=(130, 130), dtype='<U1')

In [12]:
from typing import List
from tqdm import tqdm

DIRECTIONS = ["^", ">", "v", "<"]


def get_next_direction(direction):
    return DIRECTIONS[(DIRECTIONS.index(direction) + 1) % len(DIRECTIONS)]


def get_next_position(position, direction):
    y, x = position

    if direction == "^":
        return (y - 1, x)
    elif direction == ">":
        return (y, x + 1)
    elif direction == "v":
        return (y + 1, x)
    elif direction == "<":
        return (y, x - 1)
    else:
        raise ValueError("Invalid direction")


def run_simulation(map):
    pos_dir_history = np.zeros(map.shape + (len(DIRECTIONS),))

    guard_pos = np.where((map == "^") | (map == ">") | (map == "v") | (map == "<"))
    guard_pos = guard_pos[0][0], guard_pos[1][0]

    guard_dir = map[guard_pos]

    while True:
        # EXIT: Walked out of bounds
        if (
            guard_pos[0] < 0
            or guard_pos[0] >= len(map)
            or guard_pos[1] < 0
            or guard_pos[1] >= len(map[guard_pos[0]])
        ):
            return map, 0

        # EXIT: Got stuck in loop (visited same position and direction previously)
        if pos_dir_history[guard_pos + (DIRECTIONS.index(guard_dir),)] == 1:
            return map, 1

        pos_dir_history[guard_pos + (DIRECTIONS.index(guard_dir),)] = 1

        next_pos = get_next_position(guard_pos, guard_dir)

        # EXIT: Will walk out of bounds
        if (
            next_pos[0] < 0
            or next_pos[0] >= len(map)
            or next_pos[1] < 0
            or next_pos[1] >= len(map[next_pos[0]])
        ):
            map[guard_pos] = "X"
            return map, 0

        if map[next_pos] == "#":
            map[guard_pos] = guard_dir = get_next_direction(guard_dir)
        else:
            map[guard_pos] = "X"
            guard_pos = next_pos
            map[guard_pos] = guard_dir


new_input, looped = run_simulation(input.copy())

["".join(row) for row in new_input]

['..........#...X......#.#.......................................................................#...............#......#...#.......',
 '..........XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX#..#..........#.........#..#..............................................................',
 '.........#X...X........................X...........................XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX#................',
 '..........X...X...................#.#..X...........................X...#..#.......#.....#.#............#.#......X.................',
 '...#......X...X........................X...................#.......X.#...................#XXXXXXXXXXXXXXXXX#....X.................',
 '..........X...X#.........#...........#.X...........................X..................#...X..........#....X.....X#.............#..',
 '..........X...X....#..#..XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX#X...............X.....X.................',
 '........##X...X..........X.............X##...........

In [13]:
path_pieces = np.count_nonzero((new_input == "X"))

path_pieces

4559

# Part Two

In [14]:
possible_loop_positions = 0

for idx in tqdm(np.ndindex(input.shape), total=np.prod(input.shape), desc="Processing"):
    if input[idx] == "#" or input[idx] == "^":
        continue

    input_copy = input.copy()
    input_copy[idx] = "#"

    possible_loop_positions += run_simulation(input_copy)[1]

possible_loop_positions

Processing: 100%|██████████| 16900/16900 [01:43<00:00, 163.66it/s]


1604